In [1]:
import numpy as np
import torch

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [3]:
# Define model
model = torch.nn.Linear(3, 2)
model.weight.data = torch.abs(model.weight.data)
model.bias.data = torch.abs(model.bias.data)
list(model.parameters())

[Parameter containing:
 tensor([[0.1829, 0.4536, 0.2487],
         [0.2005, 0.4834, 0.1289]], requires_grad=True),
 Parameter containing:
 tensor([0.2375, 0.3030], requires_grad=True)]

In [4]:
train_ds = torch.utils.data.TensorDataset(inputs, targets)

In [5]:
train_dl = torch.utils.data.DataLoader(train_ds, 5, shuffle=False)

In [6]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [7]:
# Define loss function
loss_fn = torch.nn.functional.mse_loss
loss = loss_fn(model(inputs), targets)
loss.backward()
opt.step()
opt.zero_grad()
print(loss)

tensor(799.2987, grad_fn=<MseLossBackward>)


In [8]:
pred = model(inputs)
            
# 2. Calculate loss
loss = loss_fn(pred, targets)
loss


tensor(642.8423, grad_fn=<MseLossBackward>)

In [9]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
      
    return torch.sqrt(loss)

In [10]:
print(torch.sqrt(torch.nn.functional.mse_loss(model(inputs), targets)))
fit(1000, model, loss_fn, opt, train_dl)
print(torch.sqrt(torch.nn.functional.mse_loss(model(inputs), targets)))

tensor(25.3543, grad_fn=<SqrtBackward>)
Epoch [10/1000], Loss: 233.6105194091797
Epoch [20/1000], Loss: 165.03628540039062
Epoch [30/1000], Loss: 118.16915130615234
Epoch [40/1000], Loss: 86.05180358886719
Epoch [50/1000], Loss: 63.9661865234375
Epoch [60/1000], Loss: 48.704811096191406
Epoch [70/1000], Loss: 38.08769989013672
Epoch [80/1000], Loss: 30.633386611938477
Epoch [90/1000], Loss: 25.335586547851562
Epoch [100/1000], Loss: 21.51076889038086
Epoch [110/1000], Loss: 18.694814682006836
Epoch [120/1000], Loss: 16.57268714904785
Epoch [130/1000], Loss: 14.930500984191895
Epoch [140/1000], Loss: 13.622955322265625
Epoch [150/1000], Loss: 12.551454544067383
Epoch [160/1000], Loss: 11.6488618850708
Epoch [170/1000], Loss: 10.869513511657715
Epoch [180/1000], Loss: 10.182209968566895
Epoch [190/1000], Loss: 9.565546035766602
Epoch [200/1000], Loss: 9.004712104797363
Epoch [210/1000], Loss: 8.489391326904297
Epoch [220/1000], Loss: 8.012310981750488
Epoch [230/1000], Loss: 7.5682249069

In [11]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 56.9973,  70.5008],
        [ 81.8621, 100.3011],
        [118.4549, 133.3354],
        [ 20.8797,  37.9084],
        [101.5785, 118.0455],
        [ 55.7461,  69.4149],
        [ 81.6925, 100.3504],
        [118.7324, 133.9092],
        [ 22.1309,  38.9943],
        [102.6601, 119.1808],
        [ 56.8277,  70.5501],
        [ 80.6109,  99.2152],
        [118.6246, 133.2861],
        [ 19.7981,  36.7732],
        [102.8298, 119.1314]], grad_fn=<AddmmBackward>)

In [12]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [13]:
torch.sqrt(torch.mean((preds - targets) * (preds - targets)))

tensor(1.0604, grad_fn=<SqrtBackward>)